In [256]:
import pandas as pd
import json
import re
from string import punctuation
from konlpy.tag import Mecab
import datetime
import pickle
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')
# mecab = Mecab()

In [2]:
TIME_MAPPING = {}
TIME_MAPPING[0] = (datetime.datetime(1, 1, 1, 16, 00), datetime.datetime(1, 1, 2, 8, 59)) # 하루 전 조건 넣어줘야함
i = 1
START = datetime.datetime(1,1,2,9,0)

s = START
while s.strftime('%H:%M') < "16:00":
    e = s + datetime.timedelta(minutes = 4)
    TIME_MAPPING[i] = (s, e)

    s += datetime.timedelta(minutes=5)
    i += 1
# TIME_MAPPING[len(TIME_MAPPING)] = (s, )

In [3]:
# len(TIME_MAPPING)
# TIME_MAPPING[86]
TIME_MAPPING
# s

{0: (datetime.datetime(1, 1, 1, 16, 0), datetime.datetime(1, 1, 2, 8, 59)),
 1: (datetime.datetime(1, 1, 2, 9, 0), datetime.datetime(1, 1, 2, 9, 4)),
 2: (datetime.datetime(1, 1, 2, 9, 5), datetime.datetime(1, 1, 2, 9, 9)),
 3: (datetime.datetime(1, 1, 2, 9, 10), datetime.datetime(1, 1, 2, 9, 14)),
 4: (datetime.datetime(1, 1, 2, 9, 15), datetime.datetime(1, 1, 2, 9, 19)),
 5: (datetime.datetime(1, 1, 2, 9, 20), datetime.datetime(1, 1, 2, 9, 24)),
 6: (datetime.datetime(1, 1, 2, 9, 25), datetime.datetime(1, 1, 2, 9, 29)),
 7: (datetime.datetime(1, 1, 2, 9, 30), datetime.datetime(1, 1, 2, 9, 34)),
 8: (datetime.datetime(1, 1, 2, 9, 35), datetime.datetime(1, 1, 2, 9, 39)),
 9: (datetime.datetime(1, 1, 2, 9, 40), datetime.datetime(1, 1, 2, 9, 44)),
 10: (datetime.datetime(1, 1, 2, 9, 45), datetime.datetime(1, 1, 2, 9, 49)),
 11: (datetime.datetime(1, 1, 2, 9, 50), datetime.datetime(1, 1, 2, 9, 54)),
 12: (datetime.datetime(1, 1, 2, 9, 55), datetime.datetime(1, 1, 2, 9, 59)),
 13: (datetim

# pororo 라벨링, 1차 단어사전 만들기

In [ ]:
%%writefile naver_pickle.py
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

for f in os.listdir('Naver종토방')[1:]:
  NAVER = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'views', 'pos', 'neg', 'nsmc', 'shop'])
  if f.endswith('json'):
    print(f)
    for line in tqdm(open('Naver종토방/'+ f, 'r', encoding = 'utf8').readlines()):
      j = json.loads(line)
      nm_sa = s_nm(j['title'], show_probs = True)['positive']
      sh_sa = s_sh(j['title'], show_probs = True)['positive']
      j.update(nsmc = nm_sa, shop = sh_sa),
      NAVER = NAVER.append(j, ignore_index = True)
      NAVER.to_pickle(f"./naver_all_{f}.pkl")

  # pd.read_pickle('./naver_all.pkl').append(pd.read_pickle('./naver_all2.pkl'), ignore_index= True).to_pickle('./naver_all.pkl')

In [ ]:
%%writefile dc_pickle.py
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

for f in os.listdir('DC갤러리'):
  DC = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'views', 'pos', 'neg', 'nsmc', 'shop'])
  if f.endswith('json'):
    print(f)
    for line in tqdm(open('DC갤러리/'+ f, 'r', encoding = 'utf8').readlines()):
      j = json.loads(line)
      nm_sa = s_nm(j['title'], show_probs = True)['positive']
      sh_sa = s_sh(j['title'], show_probs = True)['positive']
      j.update(nsmc = nm_sa, shop = sh_sa)
      DC = DC.append(j, ignore_index = True)
      DC.to_pickle(f"./dc_all_{f}.pkl")

In [ ]:
# 백그라운드로 파일 실행
# !nohup /usr/bin/python3 naver_pickle.py > naver.log 2>&1 &
# !nohup /usr/bin/python3 dc_pickle.py > dc.log 2>&1 &

# log 확인
# !tail -f naver.log
# !tail -f dc.log

# 감성사전 만들기

mecab 체ㄴ, 용ㅓ, 감탄ㅏ(확ㅣ), (관형ㅏ)
{토큰 : 감ㅓ확률, 토ㄴ: 감ㅓ확률}


In [ ]:
mecab.tagset

In [5]:
def data_preprocessing(pkl : str):
  df = pd.read_pickle(pkl)
  types = {i : 'int16' for i in df.columns if i not in ['_id', 'code', 'date', 'title', 'nsmc', 'shop']}
  df = df.astype(dtype = types)
  df.date = pd.to_datetime(df.date)
  # 중복 제거
  df.drop_duplicates(subset = ['code', 'title'], inplace=True, ignore_index= True)
  df['positive'] = (df.nsmc + df.shop) / 2
  # 전처리
  # Stopwords 처리
  pattern1 = re.compile(r'[{}]'.format(re.escape(punctuation)))  # punctuation 제거
  pattern2 = re.compile(r'[^가-힣! ]')  # 특수문자, 자음, 모음, 숫자, 영어 제거
  pattern3 = re.compile(r'\s{2,}')  # white space 1개로 바꾸기.

  df.title = df.title.map(lambda x: pattern3.sub(' ',
                                                    pattern2.sub('',
                                                                    pattern1.sub('', x))))
  global mecab
  mecab = Mecab()
  # mecab 토큰화
  # df['mecab'] = df.title.apply(lambda x: mecab.morphs(x))
  df['mecab'] = df.title.apply(lambda x: mecab.pos(x))
  # N(체언), V(용언), IC(감탄사)만 거르기
  for m in df.mecab.values:
    for w, t in m:
        # print(m)
        if t.startswith('N') or t.startswith("V") or t == 'IC': pass
        else: m.remove((w, t))
  
  return df

In [6]:
NAVER = data_preprocessing('./naver_all.pkl').rename(columns = {'views':'view'})
DC = data_preprocessing('./dc_all.pkl')

In [61]:
# naver = NAVER.drop(columns = ['_id', 'nsmc', 'shop'])
# DATA = NAVER[['code', 'date', 'mecab', 'view', 'positive']].append(DC[['code', 'date', 'mecab', 'view', 'positive']],
                                                                    # ignore_index = True)
DATA = NAVER.drop(columns=['_id', 'pos', 'neg']).append(DC.drop(columns = ['recommend', '_id']), ignore_index = True)
DATA[['code', 'date', 'title', 'mecab', 'view', 'positive']]

,code,date,title,mecab,view,positive
0,008770,2021-10-02 18:54:00,솔직히,[],5,0.030165
1,008770,2021-10-02 14:44:00,애 호텔주 아니라니까,"[(애, NNG), (호텔, NNG), (주, NNG), (아니, VCN)]",116,0.057826
2,008770,2021-10-02 13:04:00,솔브레인 화요일 시초에 담아라,"[(솔, NNP), (브레인, NNP), (화요일, NNG), (시초, NNG), ...",204,0.856793
3,008770,2021-10-02 12:04:00,기대가 크면 실망이 크다,"[(기대, NNG), (크, VA), (실망, NNG), (크, VA)]",272,0.011657
4,008770,2021-10-02 10:20:00,기가막히게 쌍바닥이네,"[(기, NNG), (막히, VV), (쌍, NNG), (바닥, NNG), (이, ...",209,0.022041
...,...,...,...,...,...,...
214170,041140,2022-01-03 10:33:37,저번주에 넥슨지티 만지작 햇는데,"[(저번, NNG), (주, NNG), (넥슨, NNP), (지티, NNP), (햇...",76,0.242665
214171,041140,2022-01-03 10:13:53,넥슨지티감사합니다,"[(넥슨, NNP), (지티, NNP), (감사, NNG)]",66,0.502039
214172,041140,2022-01-03 10:13:36,넥슨지티 물린거 개같이 탈출,"[(넥슨, NNP), (지티, NNP), (물린, VV+ETM), (거, NNB),...",169,0.074371
214173,041140,2022-01-02 14:19:26,넷게임즈 넥슨지티중에 뭐사야됨,"[(넷, IC), (게임즈, NNP), (넥슨, NNP), (지티, NNP), (중...",123,0.563175


In [10]:
DATA.to_pickle("./DATA.pkl")

In [11]:
SENTIMENT = {}

In [12]:
for data in DATA[['mecab', 'view', 'positive']].values:
    # print(data[0])
    for w, t in data[0]: # mecab tokens
        if w in SENTIMENT.keys():
            SENTIMENT[w].append((data[1], data[2]))
        else:
            SENTIMENT[w] = [(data[1], data[2])]
    # break

In [21]:
SENTIMENT['넥슨']

[(170, 0.35563676059246063),
 (120, 0.3136954978108406),
 (803, 0.11383883841335773),
 (419, 0.13565491791814566),
 (234, 0.1426195427775383),
 (428, 0.1957569718360901),
 (113, 0.9835244119167328),
 (187, 0.08162579871714115),
 (371, 0.026991701684892178),
 (781, 0.050540365278720856),
 (111, 0.09827875532209873),
 (358, 0.4869778901338577),
 (449, 0.2586163282394409),
 (131, 0.17834227345883846),
 (455, 0.12103454349562526),
 (292, 0.047839175909757614),
 (297, 0.09750085324048996),
 (91, 0.30589422583580017),
 (133, 0.045918452204205096),
 (94, 0.5181725770235062),
 (197, 0.18071159534156322),
 (451, 0.09602127224206924),
 (327, 0.004477505222894251),
 (300, 0.24535368755459785),
 (270, 0.28139903023838997),
 (393, 0.2213432490825653),
 (259, 0.36555638164281845),
 (723, 0.07803044095635414),
 (123, 0.2631223425269127),
 (303, 0.1254799049347639),
 (280, 0.08965968899428844),
 (160, 0.5246068090200424),
 (379, 0.0753401224501431),
 (527, 0.046431371942162514),
 (448, 0.0414217300713

In [22]:
for key, values in SENTIMENT.items():
    one_view = 0
    one_positive = 0
    for v in values: one_view += v[0]
    for v in values: one_positive += (v[0] / one_view) * v[1]

    SENTIMENT[key] = one_positive

조회수 가중치(token별)  
  
    
$\frac{v_1}{v_1 + ... + v_n} * positive_1 + \frac{v_2}{v_1 + ... + v_n} * positive_2 + ... + \frac{v_n}{v_1 + ... + v_n} * positive_n = POSITIVE_{token} $

In [302]:
pd.DataFrame.from_dict(SENTIMENT, orient= 'index').reset_index().rename(columns = {'index': 'token', 0:'positive'})

# SENTIMENT

,token,positive
0,애,0.261986
1,호텔,0.336487
2,주,0.313494
3,아니,0.154926
4,솔,0.292908
...,...,...
31332,양기,0.240577
31333,면도기,0.077069
31334,세척,0.077069
31335,악한,0.294283


In [24]:
# json.dump(SENTIMENT, open('./SENTIMENT_ver1.0.json', 'w'), ensure_ascii=False)
pickle.dump(SENTIMENT, open('SENTIMENT_ver1.0.pkl', 'wb'))

In [25]:
# json.load(open('./SENTIMENT_ver1.0.json', 'r'))
SENTIMENT = pickle.load(open('SENTIMENT_ver1.0.pkl', 'rb'))
# SENTIMENT

# STOCK_PRICE 5분단위 & time_mapping

In [26]:
import sqlite3
conn = sqlite3.connect('/Users/yejin/Desktop/Career/Projects/AYOA/Stock.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM stock")
STOCK = pd.DataFrame(columns = ['pk', 'code', 'date', 'time', 'price'], data = cursor.fetchall()).drop(columns = 'pk')

In [27]:
STOCK.head(10)

,code,date,time,price
0,000660,20210820,09:00,"103,000"
1,000660,20210820,09:01,"103,000"
2,000660,20210820,09:02,"103,000"
3,000660,20210820,09:03,"103,500"
4,000660,20210820,09:04,"103,500"
5,000660,20210820,09:05,"103,000"
6,000660,20210820,09:06,"103,500"
7,000660,20210820,09:07,"103,500"
8,000660,20210820,09:08,"103,000"
9,000660,20210820,09:09,"103,500"


In [28]:
STOCK_PRICE = pd.DataFrame(columns = ['code', 'date', 'time', 'price'])

for i in tqdm(range(len(STOCK))):
    if int(STOCK.time[i][-2:]) % 5 == 0:
        STOCK_PRICE = STOCK_PRICE.append(STOCK.iloc[i], ignore_index=True)
del STOCK

100%|██████████| 527190/527190 [08:21<00:00, 1050.31it/s]


In [29]:
STOCK_PRICE

,code,date,time,price
0,000660,20210820,09:00,"103,000"
1,000660,20210820,09:05,"103,000"
2,000660,20210820,09:10,"102,500"
3,000660,20210820,09:15,"102,500"
4,000660,20210820,09:20,"102,000"
...,...,...,...,...
108347,230980,20211108,09:05,"1,905"
108348,230980,20211108,09:00,"1,890"
108349,230980,20211108,09:10,"1,900"
108350,230980,20211108,09:05,"1,905"


In [30]:
STOCK_PRICE.to_pickle('./STOCK.pkl')

In [143]:
STOCK_PRICE = pd.read_pickle('./STOCK.pkl')
STOCK_PRICE.date = STOCK_PRICE.date + " " + STOCK_PRICE.time
STOCK_PRICE.date = STOCK_PRICE.date.astype('datetime64')
STOCK_PRICE.price = STOCK_PRICE.price.str.replace(',', "").astype('int')

for i in tqdm(range(len(STOCK_PRICE))):
    for k in range(86):
        if STOCK_PRICE.date[i].time() == TIME_MAPPING[k][0].time():
            STOCK_PRICE.loc[i, 'time_map'] = k
            break
STOCK_PRICE.to_pickle('./STOCK_PRICE_map.pkl')

100%|██████████| 108352/108352 [00:57<00:00, 1869.88it/s]


In [ ]:
del STOCK

# mecab_데이터 time_mapping

In [144]:
DATA = pd.read_pickle('./DATA.pkl')
# DATA

In [33]:
DATA.head()

,code,date,title,view,nsmc,shop,positive,mecab
0,008770,2021-10-02 18:54:00,솔직히,5,0.039333,0.020997,0.030165,[]
1,008770,2021-10-02 14:44:00,애 호텔주 아니라니까,116,0.065854,0.049799,0.057826,"[(애, NNG), (호텔, NNG), (주, NNG), (아니, VCN)]"
2,008770,2021-10-02 13:04:00,솔브레인 화요일 시초에 담아라,204,0.891313,0.822273,0.856793,"[(솔, NNP), (브레인, NNP), (화요일, NNG), (시초, NNG), ..."
3,008770,2021-10-02 12:04:00,기대가 크면 실망이 크다,272,0.010065,0.013250,0.011657,"[(기대, NNG), (크, VA), (실망, NNG), (크, VA)]"
4,008770,2021-10-02 10:20:00,기가막히게 쌍바닥이네,209,0.011541,0.032541,0.022041,"[(기, NNG), (막히, VV), (쌍, NNG), (바닥, NNG), (이, ..."


In [145]:
DATA['time_map'] = 0 # 기본 0
TIME_MAPPING[85] = (datetime.datetime(1, 1, 1, 16, 0), ) # for loop

for i in tqdm(range(len(DATA))):
    for k in range(1,85):
        if DATA.date[i].time() >= TIME_MAPPING[k][0].time() and DATA.date[i].time() < TIME_MAPPING[k+1][0].time():
            DATA.loc[i, 'time_map'] = k
            break
        # else: DATA.loc[i, 'time_map'] = 0

100%|██████████| 214175/214175 [03:28<00:00, 1026.80it/s]


In [146]:
DATA.to_pickle("./DATA_map.pkl")

In [147]:
# DATA.time_map.astype('int16')
DATA = pd.read_pickle('./DATA_map.pkl')
DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214175 entries, 0 to 214174
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   code      214175 non-null  object        
 1   date      214175 non-null  datetime64[ns]
 2   title     214175 non-null  object        
 3   view      214175 non-null  int16         
 4   nsmc      214175 non-null  float64       
 5   shop      214175 non-null  float64       
 6   positive  214175 non-null  float64       
 7   mecab     214175 non-null  object        
 8   time_map  214175 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int16(1), int64(1), object(3)
memory usage: 13.5+ MB


In [148]:
# NEW_DATA = DATA[DATA.time_map != 0]
# NEW_DATA = DATA[DATA.time_map.notnull()].reset_index(drop = True)
# 9시에 8:55~8:59 데이터 가져오니까
# NEW_DATA.time_map += 1
del DATA

# 주가 등락 적용 업데이트
- 10:03 게시글 -> 긍정확률에 수식 적용하려면 % 올랐는지가 있어야 하지 -> 10시 5분에 비해 10:10분 기반으로 올랐는지 내렸는지 학습
- 18:50 게시글 -> 9:00에 비해 9:05가 올랐는지, 09:05 기반으로 학습
- pdf -> 하루종일 영향을 준다. (안 올라오는 경우가 더 많음. 올라올 때만 추가적인 정보로) +a


### 등락 수식 (감성사전 업데이트 식)
- n% 변화, $x_t = t시점의 단어 감성점수$
- 시간별로 업데이트
- known에 대해서 점수 업데이트 > 문장의 감성점수를 구할 수 있음(평균) > unknown initialize
    - known 부정/긍정 :  $x_{t+1} = x_t + n * x_t * max(해당 분위 단어들의 감성점수)$ > $min(max(x_{t+1}, 0), 1)$ > 짜잔

ex. 삼성에 대해 100개 리뷰(9시 0분 ~ 9시 4분)  
-> 100개 리뷰 전체 word counting > 30% 30% 40% 나누고 > 해당 분위 감성점수를 더 곱해줌(위 수식)  
-> a 50번 b 2번의 경우

---------
### 종목별 확률 (inference 식) ; 예측할 때
- 리뷰의 감성점수 도출: 토큰화된 리뷰의 감성 점수 평균 = 한 리뷰의 감성 점수
- 가중치 :  
- pdf 점수랑 합치는 수식 : 알파 = 0.3 예상

## 기간 맞추기

In [149]:
OPINION = pd.read_pickle("./DATA_map.pkl")
STOCK_PRICE = pd.read_pickle('./STOCK_PRICE_map.pkl')

In [151]:
STOCK_PRICE['date_time'] = STOCK_PRICE.date
STOCK_PRICE['time'] = STOCK_PRICE.date.apply(lambda x: x.time())
STOCK_PRICE.date = STOCK_PRICE.date.apply(lambda x: x.date()) 

OPINION['date_time'] = OPINION.date
OPINION['time'] = OPINION.date.apply(lambda x: x.time())
OPINION['date'] = OPINION.date.apply(lambda x: x.date())

In [205]:
DATA = OPINION.drop(columns = ['title', 'nsmc', 'shop', 'time'])
DATA.time_map += 1
# 10시 3분 게시글? 10시 5분 주가에 영향
# 장 마감 이후 게시글? 14시 이후는 다음날 9시, 08시 이전은 당일 9시
for i in tqdm(range(len(DATA))):
    if DATA.date_time[i].time() >= datetime.time(16, 0):
        DATA.loc[i, 'price_time'] = datetime.datetime.combine((DATA.date_time[i] + datetime.timedelta(days = 1)).date(),datetime.time(9,0))
    elif DATA.date_time[i].time() < datetime.time(9, 0):
       DATA.loc[i, 'price_time'] = datetime.datetime.combine(DATA.date_time[i].date(), datetime.time(9,0))
    else:
        DATA.loc[i, 'price_time'] = datetime.datetime.combine(DATA.date_time[i].date(), TIME_MAPPING[DATA.time_map[i]][0].time())
DATA

100%|██████████| 214175/214175 [05:41<00:00, 626.58it/s]


,code,date,view,positive,mecab,time_map,date_time,price_time
0,008770,2021-10-02,5,0.030165,[],1,2021-10-02 18:54:00,2021-10-03 09:00:00
1,008770,2021-10-02,116,0.057826,"[(애, NNG), (호텔, NNG), (주, NNG), (아니, VCN)]",70,2021-10-02 14:44:00,2021-10-02 14:45:00
2,008770,2021-10-02,204,0.856793,"[(솔, NNP), (브레인, NNP), (화요일, NNG), (시초, NNG), ...",50,2021-10-02 13:04:00,2021-10-02 13:05:00
3,008770,2021-10-02,272,0.011657,"[(기대, NNG), (크, VA), (실망, NNG), (크, VA)]",38,2021-10-02 12:04:00,2021-10-02 12:05:00
4,008770,2021-10-02,209,0.022041,"[(기, NNG), (막히, VV), (쌍, NNG), (바닥, NNG), (이, ...",18,2021-10-02 10:20:00,2021-10-02 10:25:00
...,...,...,...,...,...,...,...,...
214170,041140,2022-01-03,76,0.242665,"[(저번, NNG), (주, NNG), (넥슨, NNP), (지티, NNP), (햇...",20,2022-01-03 10:33:37,2022-01-03 10:35:00
214171,041140,2022-01-03,66,0.502039,"[(넥슨, NNP), (지티, NNP), (감사, NNG)]",16,2022-01-03 10:13:53,2022-01-03 10:15:00
214172,041140,2022-01-03,169,0.074371,"[(넥슨, NNP), (지티, NNP), (물린, VV+ETM), (거, NNB),...",16,2022-01-03 10:13:36,2022-01-03 10:15:00
214173,041140,2022-01-02,123,0.563175,"[(넷, IC), (게임즈, NNP), (넥슨, NNP), (지티, NNP), (중...",65,2022-01-02 14:19:26,2022-01-02 14:20:00


In [206]:
STOCK_PRICE

,code,date,time,price,time_map,date_time
0,000660,2021-08-20,09:00:00,103000,1.0,2021-08-20 09:00:00
1,000660,2021-08-20,09:05:00,103000,2.0,2021-08-20 09:05:00
2,000660,2021-08-20,09:10:00,102500,3.0,2021-08-20 09:10:00
3,000660,2021-08-20,09:15:00,102500,4.0,2021-08-20 09:15:00
4,000660,2021-08-20,09:20:00,102000,5.0,2021-08-20 09:20:00
...,...,...,...,...,...,...
108347,230980,2021-11-08,09:05:00,1905,2.0,2021-11-08 09:05:00
108348,230980,2021-11-08,09:00:00,1890,1.0,2021-11-08 09:00:00
108349,230980,2021-11-08,09:10:00,1900,3.0,2021-11-08 09:10:00
108350,230980,2021-11-08,09:05:00,1905,2.0,2021-11-08 09:05:00


In [207]:
DATA = DATA.drop(columns = ['date', 'time_map']).merge(STOCK_PRICE.drop(columns = ['date', 'time_map', 'time']).rename(columns = {"date_time": "price_time"}), on = ['code', 'price_time'])

# time_pred = 예측할 가격 시간 (15:10), price_pred = 예측할 가격 >  change : price & price_prd pct_change()
DATA['time_pred'] = DATA.price_time + datetime.timedelta(minutes = 5)
DATA = DATA.merge(STOCK_PRICE[['code', 'date_time', 'price']].rename(columns = {"date_time": 'time_pred'}), 
        on = ['code', 'time_pred']).rename(columns = {'price_x' : 'price', 'price_y': 'price_pred'})


DATA

,code,view,positive,mecab,date_time,price_time,price,time_pred,price_pred
0,008770,795,0.286026,"[(호텔신라, NNP), (오늘, MAG), (빠지, VV), (거, NNB), (...",2021-10-01 15:00:00,2021-10-01 15:05:00,85000,2021-10-01 15:10:00,84800
1,008770,442,0.619571,"[(담, VV), (담, VV)]",2021-10-01 14:59:00,2021-10-01 15:00:00,85000,2021-10-01 15:05:00,85000
2,008770,267,0.018406,"[(뭐, NP), (야, VCP+EF)]",2021-10-01 13:41:00,2021-10-01 13:45:00,84600,2021-10-01 13:50:00,84600
3,008770,321,0.186608,"[(오늘, NNG), (추매, NNG)]",2021-10-01 13:11:00,2021-10-01 13:15:00,85100,2021-10-01 13:20:00,85100
4,008770,224,0.075681,"[(에라, IC), (모텔, NNG), (도, JX), (못한, VX+ETM), (...",2021-10-01 12:56:00,2021-10-01 13:00:00,85200,2021-10-01 13:05:00,85200
...,...,...,...,...,...,...,...,...,...
124124,069410,166,0.011228,"[(메타, NNG), (버스, NNG), (사기, VV)]",2021-09-14 17:53:32,2021-09-15 09:00:00,11300,2021-09-15 09:05:00,11150
124125,069410,166,0.011228,"[(메타, NNG), (버스, NNG), (사기, VV)]",2021-09-14 17:53:32,2021-09-15 09:00:00,11300,2021-09-15 09:05:00,11150
124126,069410,166,0.011228,"[(메타, NNG), (버스, NNG), (사기, VV)]",2021-09-14 17:53:32,2021-09-15 09:00:00,11300,2021-09-15 09:05:00,11150
124127,069410,166,0.011228,"[(메타, NNG), (버스, NNG), (사기, VV)]",2021-09-14 17:53:32,2021-09-15 09:00:00,11300,2021-09-15 09:05:00,11150


In [208]:
DATA['change'] = (DATA['price_pred'] - DATA.price) / DATA.price_pred * 100
DATA

,code,view,positive,mecab,date_time,price_time,price,time_pred,price_pred,change
0,008770,795,0.286026,"[(호텔신라, NNP), (오늘, MAG), (빠지, VV), (거, NNB), (...",2021-10-01 15:00:00,2021-10-01 15:05:00,85000,2021-10-01 15:10:00,84800,-0.235849
1,008770,442,0.619571,"[(담, VV), (담, VV)]",2021-10-01 14:59:00,2021-10-01 15:00:00,85000,2021-10-01 15:05:00,85000,0.000000
2,008770,267,0.018406,"[(뭐, NP), (야, VCP+EF)]",2021-10-01 13:41:00,2021-10-01 13:45:00,84600,2021-10-01 13:50:00,84600,0.000000
3,008770,321,0.186608,"[(오늘, NNG), (추매, NNG)]",2021-10-01 13:11:00,2021-10-01 13:15:00,85100,2021-10-01 13:20:00,85100,0.000000
4,008770,224,0.075681,"[(에라, IC), (모텔, NNG), (도, JX), (못한, VX+ETM), (...",2021-10-01 12:56:00,2021-10-01 13:00:00,85200,2021-10-01 13:05:00,85200,0.000000
...,...,...,...,...,...,...,...,...,...,...
124124,069410,166,0.011228,"[(메타, NNG), (버스, NNG), (사기, VV)]",2021-09-14 17:53:32,2021-09-15 09:00:00,11300,2021-09-15 09:05:00,11150,-1.345291
124125,069410,166,0.011228,"[(메타, NNG), (버스, NNG), (사기, VV)]",2021-09-14 17:53:32,2021-09-15 09:00:00,11300,2021-09-15 09:05:00,11150,-1.345291
124126,069410,166,0.011228,"[(메타, NNG), (버스, NNG), (사기, VV)]",2021-09-14 17:53:32,2021-09-15 09:00:00,11300,2021-09-15 09:05:00,11150,-1.345291
124127,069410,166,0.011228,"[(메타, NNG), (버스, NNG), (사기, VV)]",2021-09-14 17:53:32,2021-09-15 09:00:00,11300,2021-09-15 09:05:00,11150,-1.345291


In [218]:
data = DATA.copy()
data.mecab = data.mecab.astype("str")
dup_idx = data[data.duplicated()].index

In [220]:
DATA = DATA.drop(index = dup_idx).reset_index(drop = True)

In [221]:
DATA

,code,view,positive,mecab,date_time,price_time,price,time_pred,price_pred,change
0,008770,795,0.286026,"[(호텔신라, NNP), (오늘, MAG), (빠지, VV), (거, NNB), (...",2021-10-01 15:00:00,2021-10-01 15:05:00,85000,2021-10-01 15:10:00,84800,-0.235849
1,008770,442,0.619571,"[(담, VV), (담, VV)]",2021-10-01 14:59:00,2021-10-01 15:00:00,85000,2021-10-01 15:05:00,85000,0.000000
2,008770,267,0.018406,"[(뭐, NP), (야, VCP+EF)]",2021-10-01 13:41:00,2021-10-01 13:45:00,84600,2021-10-01 13:50:00,84600,0.000000
3,008770,321,0.186608,"[(오늘, NNG), (추매, NNG)]",2021-10-01 13:11:00,2021-10-01 13:15:00,85100,2021-10-01 13:20:00,85100,0.000000
4,008770,224,0.075681,"[(에라, IC), (모텔, NNG), (도, JX), (못한, VX+ETM), (...",2021-10-01 12:56:00,2021-10-01 13:00:00,85200,2021-10-01 13:05:00,85200,0.000000
...,...,...,...,...,...,...,...,...,...,...
86482,030350,43,0.139713,"[(드래곤, NNP), (플라이, NNP), (광고, NNG), (번, NNBC),...",2021-08-31 14:53:03,2021-08-31 14:55:00,2810,2021-08-31 15:00:00,2625,-7.047619
86483,030350,80,0.769799,"[(드래곤, NNP), (플라이, NNP), (타, VV), (이거, NP), (더...",2021-08-25 10:25:22,2021-08-25 10:30:00,2590,2021-08-25 10:35:00,2595,0.192678
86484,069410,324,0.315969,"[(쇼박스, NNP), (한, XSA+ETM), (나라, NNG), (엔, NNG)...",2021-09-27 01:26:49,2021-09-27 09:00:00,10600,2021-09-27 09:05:00,10500,-0.952381
86485,069410,166,0.011228,"[(메타, NNG), (버스, NNG), (사기, VV)]",2021-09-14 17:53:32,2021-09-15 09:00:00,11300,2021-09-15 09:05:00,11150,-1.345291


In [222]:
DATA.to_pickle("./DATA_change.pkl")

## 등락수식 적용

In [ ]:
DATA = pd.read_pickle('./DATA_change.pkl')

In [224]:
# 종목별 분위 나누기
TICKER = DATA.code.unique()
# TICKER

array(['008770', '336260', '009830', '086520', '011930', '018000',
       '035720', '035420', '036570', '018260', '005930', '000660',
       '011790', '014680', '000990', '023530', '004170', '028260',
       '026960', '105560', '055550', '316140', '086790', '041140',
       '089890', '030350', '230980', '071050', '047080', '069410'],
      dtype=object)

In [271]:
from collections import Counter
# wc를 map-reduce로? 과한듯
for i in tqdm(range(len(DATA))):
    DATA.mecab[i] = [(w, DATA.change[i]) for w, _ in DATA.mecab[i]]

100%|██████████| 86487/86487 [01:09<00:00, 1243.35it/s]


In [ ]:
SENTIMENT = pickle.load(open('SENTIMENT_ver1.0.pkl', 'rb'))

In [340]:
# 5분 단위 투자자 반응
one_term = DATA[DATA.price_time == DATA.price_time.unique()[0]]
for t in TICKER:
# t = TICKER[0]
tokens = one_term[one_term.code == t].mecab.sum()
# if len(tokens) == 0 : break
word_count = Counter([i[0] for i in tokens]).most_common()
# 분위 나누기
r1 = [i[0] for i in word_count[:int(len(word_count) * .3)]]
r2 = [i[0] for i in word_count[int(len(word_count) * .3):int(len(word_count) * .6)]]
r3 = [i[0] for i in word_count[int(len(word_count) * .6):]]
if len(r1) == 0: max_r1 = 0
else: max_r1 = max([SENTIMENT[i] for i in r1])
if len(r2) == 0: max_r2 = 0
else: max_r2 = max([SENTIMENT[i] for i in r2])
if len(r3) == 0: max_r1 = 0
else: max_r3 = max([SENTIMENT[i] for i in r3])

# 단어의 여러 change 값 평균 내기
token_df = pd.DataFrame(tokens, columns = ['token', 'change'])
token_df = token_df.groupby('token').mean()['change']

# 감성사전 업데이트
for one in r1: SENTIMENT[one] = min(max(SENTIMENT[one] * (1 + max_r1 * token_df[one]), 0), 1)
for one in r2: SENTIMENT[one] = min(max(SENTIMENT[one] * (1 + max_r2 * token_df[one]), 0), 1)
for one in r3: SENTIMENT[one] = min(max(SENTIMENT[one] * (1 + max_r3 * token_df[one]), 0), 1)

In [341]:
one = r1[0]
SENTIMENT[one] * (1 + max_r1 * token_df[one])

0.31109622240070556

In [342]:
SENTIMENT[one]

0.33804825625916335

In [339]:
token_df = 

,change
token,
거,-0.235849
보,-0.235849
빠지,-0.235849
오늘,-0.235849
호텔신라,-0.235849
